## Text Based Emotion Recognition
Samarth Tripathi et.al[1] considered concatenating each pre-trained 300-dimensional glove word vector to produce a utterance level feature for text based emotion recognition. They used a maximum sequence length of 500 to obtain a (500,300) vector for each utterance. This is a straightforward approach to provide a quick and crude utterance-vector that can often be very useful. They then fed the obtained utterance-vector to a LSTM model with the time steps the same as maximum sequence length 500 and achieved a test accuracy 64.47%. The LSTM model they had captured the information of previous words which could be very helpful in predicting the label of the whole utterance.

We consider different approaches which train the utterance-vector directly instead of concatenation word vectors since simple concatenating loses semantics of the utterance as discussed in https://arxiv.org/abs/1405.4053. We use 3 kinds of approaches to extract utterance level vector directly and compare their performance with each other with 3 layers fully connected neural network. 

We suspect that previous utterances may also add useful information to the prediction of current utterance. Instead of just capturing the memory of previous words in the same utterance as done by Samarth Tripathi et.al, we capture a memory of previous utterances using a LSTM model. We define a variable time_steps which determines the length of time series. We are going to reshape the training samples of shape (num_utterances, num_features) to (num_utterances/time_steps, time_steps, num_features). Since num_utterances may not be divisible by time_steps, we duplicate smallest needed samples from the beginning of the training set to the end of the training set to form a new training set of shape (num_utterances+needed, num_features). The reshaped training set has the shape as ((num_utterances+needed)/time_steps, time_steps, num_features). We then feed the training data through a LSTM layer with 256 hidden neurons followed by 3 fully connected layers. Detailed layer information can be found from the codes. We then compare its performance with the above fully connected neural networks.

(1) _Doc2Vec_: Doc2Vec is a powerful technique to extract utterance level vector directly introduce by Le and Mikolov https://arxiv.org/abs/1405.4053. There are 2 kinds of Doc2Vec methods, Paragraph Vector - Distributed Memory (PV-DM) and Paragraph Vector - Distributed Bag of Words (PV-DBOW). PV-DBOW also has 2 different mechanisms, averaging and concatenation. Le and Mikolov notes that combining a paragraph vector from DBOW and DM improves performance. We follow their suggestion, pairing the models together to produce 5 different models. We then run a linear svm (polynomial or gaussian provides worse result) on the training data and calculate cross validation accuracy. As we can see, the PV-DBOW(model 0) provides the best accuracy among the three single model. with PV-DBOW + PV-DM(average), the accuracy improves a little bit. Thus, we use this model to extract utterance-vector which has 500 dimensions for each utterance.<br>
(2) _Skip-Thought Vectors_: Skip-Thought vector is another powerful technique first introduced by Kiros et. al (https://arxiv.org/abs/1506.06726) to include neighbor utterances information into current utterance through unsupervised learning. They train an encoder decoder model that tries to reconstruct the surrounding utterance of an encoded utterance. Utterances that share semantic and syntactic properties are thus mapped to similar vector representations. The product of Skip-Thought vector is the encoder, while the decoder is thrown away after training. To generate the Skip-Thought vectors, we follow the approach released by ryankiros(https://github.com/ryankiros/skip-thoughts). After training, we get 4000-dimensional vector for each utterance.<br>
(3) _Sent2Vec_: This is simple unsupervised model to compose utterance embeddings using the word vectors along with n-gram embeddings and it has been shown to outperform a lot of other unsupervised models on a lot of benchmark tasks as shown in https://arxiv.org/abs/1703.02507. To generate the Sent2Vec for our model, we follow the approach released by the origin author Matteo Pagliardini (https://github.com/epfml/sent2vec). We use the pretrained model sent2vec_wiki_unigrams(https://drive.google.com/open?id=0B6VhzidiLvjSa19uYWlLUEkzX3c) 5GB (600dim, trained on English Wikipedia) released by them to generate utterance-vector. After training, we get 600-dimensional vector for each utterance. <br>

We feed the above utterance vectors obtained by 3 different feature extraction approaches to a 3 layer fully connected neural network seperately as a baseline. We then feed the utterance vectors to a LSTM with a time_steps 16 and compare the result with the baseline as shown in table 2. As we can see, a LSTM model capturing memory of previous utterances provides a huge improvement for all feature extraction approaches. For example, LSTM model increases the test accuracy from 62.15% to 69.93%.
With only text features, we improve the test accuray from 64.78% as in [1] to 70.95% with a LSTM model which is able to capture utterance level memory. <br>

|  table 1 	| PV-DBOW 	| PV-DM(concat) 	| PV-DM(average) 	| PV-DBOW + PV-DM(concat) 	| PV-DBOW + PV-DM(average) 	|
|:--------:	|:-------:	|:-------------:	|:--------------:	|:-----------------------:	|:------------------------:	|
| accuracy 	|  53.11% 	|     46.83%    	|     33.79%     	|          53.21%         	|          53.87%          	|

table 1: Cross validation accuracy for different Doc2Vec models in training dataset


|     table2   | DNN    | LSTM   |
| ------------ | ------ | ------ |
| Doc2Vec      | 62.15% | 68.11% |
| Skip-Thought | 62.15% | 69.93% |
| Sent2Vec     | 64.57% | 70.95% |

tabel 2: Test accuracy for 3 different feature extraction approaches using DNN and LSTM


## Conclusions and Future Work
An Increasing accuracy trend is observed as features from various modalities are combined. We improve the accuray from 71.04% in [1] to 76.21% by combining three features. LSTM performs better than CNN in extracting MOCAP based features when combined with other modalities. Also, We work on a neural network structure initialization strategy from random forest for stabilization. In text feature extraction, we use LSTM to capture utterances memory and improve the accuray from 64.78% as in [1] to 70.95% with only text features.

There are some future works can be done to further improve the performance of the model. One extension is to verify if utterance level memory help in improving speech and MOCAP features since it gives a huge improvement if text feature extraction. We use a time_steps 16 to form a time series sample, other time_steps can be explored and check how it influences the performance. Instead of using 16 utterance to form a time series sample, and then use a many to many LSTM model, one may also use previous 16 utterances of current utterance and build a many to one LSTM model. This may increase the accuracy further. One more extension is to find ways to convert low-level features into high-level features without truncating by using model based approaches. 